# Prediction of Hailstone sequences using Autoencoder

In [1]:
import sys
import numpy as np
import tensorflow as tf

In [2]:
sequences = np.load("hailstone_sequences.npy")

In [3]:
def split_xy(Xy):
    X = Xy[:, 0:-1]
    y = Xy[:, -1]
    
    return X, y

def my_shuffle(X, y):
    np.random.seed(42)
    Xy = np.concatenate((X, y[:, np.newaxis]), axis=1)
    np.random.shuffle(Xy)
    return split_xy(Xy)

def next_batch(X, y, it, batch_size, max_size):
    if it + batch_size > max_size:
        return X[it, :], y[it, :]
    else:
        return X[it:it+batch_size, ], y[it:it+batch_size, ]

In [4]:
X, y = split_xy(sequences)

In [5]:
print(X.shape)
print(y.shape)

(248, 5)
(248,)


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Train

In [7]:
# 5 > 4 > 3 (coding) < 4 < 6
n_inputs = sequences.shape[1]-1
n_hidden1 = n_inputs - 1
n_hidden2 = n_inputs - 2
n_hidden3 = n_hidden1
n_outputs = n_inputs + 1

learning_rate = 0.01
l2_reg = 0.0001

activation = tf.nn.elu
regularizer = tf.contrib.layers.l2_regularizer(l2_reg)
initializer = tf.contrib.layers.variance_scaling_initializer()

X = tf.placeholder(tf.float32, shape=[None, n_inputs])
X_full = tf.placeholder(tf.float32, shape=[None, n_inputs+1])

weights1_init = initializer([n_inputs, n_hidden1])
weights2_init = initializer([n_hidden1, n_hidden2])
weights3_init = initializer([n_hidden2, n_hidden3])
weights4_init = initializer([n_hidden3, n_outputs])

weights1 = tf.Variable(weights1_init, dtype=tf.float32, name="weights1")
weights2 = tf.Variable(weights2_init, dtype=tf.float32, name="weights2")
weights3 = tf.Variable(weights3_init, dtype=tf.float32, name="weights3")
weights4 = tf.Variable(weights4_init, dtype=tf.float32, name="weights4")

biases1 = tf.Variable(tf.zeros(n_hidden1), name="biases1")
biases2 = tf.Variable(tf.zeros(n_hidden2), name="biases2")
biases3 = tf.Variable(tf.zeros(n_hidden3), name="biases3")
biases4 = tf.Variable(tf.zeros(n_outputs), name="biases4")

hidden1 = activation(tf.matmul(X, weights1) + biases1)
hidden2 = activation(tf.matmul(hidden1, weights2) + biases2)
hidden3 = activation(tf.matmul(hidden2, weights3) + biases3)
outputs = tf.matmul(hidden3, weights4) + biases4

reconstruction_loss = tf.reduce_mean(tf.square(outputs - X_full))

In [8]:
optimizer = tf.train.AdamOptimizer(learning_rate)

with tf.name_scope("phase1"):
    phase1_outputs = tf.matmul(hidden1, weights4) + biases4  # bypass hidden2 and hidden3
    phase1_reconstruction_loss = tf.reduce_mean(tf.square(phase1_outputs - X_full))
    phase1_reg_loss = regularizer(weights1) + regularizer(weights4)
    phase1_loss = phase1_reconstruction_loss + phase1_reg_loss
    phase1_training_op = optimizer.minimize(phase1_loss)

with tf.name_scope("phase2"):
    phase2_reconstruction_loss = tf.reduce_mean(tf.square(hidden3 - hidden1))
    phase2_reg_loss = regularizer(weights2) + regularizer(weights3)
    phase2_loss = phase2_reconstruction_loss + phase2_reg_loss
    train_vars = [weights2, biases2, weights3, biases3]
    phase2_training_op = optimizer.minimize(phase2_loss, var_list=train_vars) # freeze hidden1

In [9]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [10]:
training_ops = [phase1_training_op, phase2_training_op]
reconstruction_losses = [phase1_reconstruction_loss, phase2_reconstruction_loss]
n_epochs = [4, 10]
batch_sizes = [10, 10]

with tf.Session() as sess:
    init.run()
    for phase in range(2):
        print("Training phase #{}".format(phase + 1))
        for epoch in range(n_epochs[phase]):
            n_batches = X_train.shape[0] // batch_sizes[phase]
            
            X_train_shuffle, y_train_shuffle = my_shuffle(X_train, y_train)
            for iteration in range(n_batches):
                print("\r{}%".format(100 * iteration // n_batches), end="")
                sys.stdout.flush()
                
#                 X_batch, y_batch = mnist.train.next_batch(batch_sizes[phase])
                X_batch, y_batch = next_batch(X_train_shuffle, y_train_shuffle, \
                                              iteration, batch_sizes[phase], X_train.shape[0])
                
                
#                 X_full_tmp = X_batch
                X_full_tmp = np.concatenate((X_batch, y_batch[:, np.newaxis]), axis=1)
                sess.run(training_ops[phase], feed_dict={X: X_batch, X_full: X_full_tmp})
                
            loss_train = reconstruction_losses[phase].eval(feed_dict={X: X_batch, X_full: X_full_tmp})
            print("\r{}".format(epoch), "Train MSE:", loss_train)
            saver.save(sess, "./hailstone_prediction.ckpt")
    loss_test = reconstruction_loss.eval(feed_dict={X: X_test, X_full: np.concatenate((X_test, y_test[:, np.newaxis]), axis=1)})
    print("Test MSE:", loss_test)

Training phase #1
0 Train MSE: 293509.0
1 Train MSE: 116483.0
2 Train MSE: 36325.9
3 Train MSE: 11693.4
Training phase #2
0 Train MSE: 211088.0
1 Train MSE: 73209.1
2 Train MSE: 42224.0
3 Train MSE: 42096.4
4 Train MSE: 39507.2
5 Train MSE: 36960.2
6 Train MSE: 36045.0
7 Train MSE: 3707.38
8 Train MSE: 1670.63
9 Train MSE: 981.541
Test MSE: 135318.0
